In [1]:

import json
import mysql.connector
import nvda_config
import requests
import pandas as pd
from datetime import datetime, timedelta

Began this page as using the original API was very limited. Phone seems to have access that I need and options for more feasible upgrade. 

In [2]:
# set up login
Cpassword = nvda_config.mysql_password
api_k = nvda_config.twelve_api_nvda
base_url = 'https://api.twelvedata.com'


In [3]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Date range for historical data (adjust as needed)
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # Get the past year's data


In [4]:
# Twelve Data API parameters
symbol = 'NVDA'
interval = '1min'  # Adjust interval if needed (e.g., '5min', '15min', etc.)
outputsize = 5000  # Maximum number of data points per request (Twelve Data limit)

# Function to fetch data in chunks (due to Twelve Data limits)
def fetch_data_chunk(start_date, end_date):
    endpoint = f'/time_series?symbol={symbol}&interval={interval}&start_date={start_date}&end_date={end_date}&outputsize={outputsize}&apikey={api_k}'
    url = base_url + endpoint
    response = requests.get(url)
    return json.loads(response.content)


In [5]:
# Fetch data in chunks
current_date = start_date
while current_date < end_date:
    chunk_end_date = current_date + timedelta(days=30)  # Fetch 30 days at a time
    tick_data = fetch_data_chunk(current_date.strftime('%Y-%m-%d'), chunk_end_date.strftime('%Y-%m-%d'))

    # Insert tick data into MySQL
    for tick in tick_data['values']:  # Note the different key 'values'
        sql = "INSERT INTO stock_ticks (symbol, time, price, volume) VALUES (%s, %s, %s, %s)"
        values = (symbol, tick['datetime'], tick['close'], tick['volume'])
        mycursor.execute(sql, values)

    current_date = chunk_end_date

mydb.commit()
#print(mycursor.rowcount, "records inserted.")